# Create Random Seed simulated data

In [ ]:
# import numpy as np

# # Define parameters
# n_indices =20000
# seed = 42
# output_file = 'index_shuffle_order.npy'

# # Generate shuffled indices with a fixed seed
# np.random.seed(seed)
# shuffled_indices = np.random.permutation(n_indices) + 1  # Indices from 1 to 20000

# # Save the shuffled indices
# np.save(output_file, shuffled_indices)

# print(f'Shuffled indices saved to {output_file}')


## Create Dataset Real Data

Find out wich indicies are in both datasets after preprocessing

In [2]:
import os
import re
import shutil

def extract_indices_from_filenames(directory):
    indices = []
    pattern = re.compile(r'(\d+)_idx')

    for filename in os.listdir(directory):
        match = pattern.search(filename)
        if match:
            index = int(match.group(1))
            indices.append(index)

    return indices

# directory_gaussian = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\phasemask_total_index_crop'
# directory_vortex = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08\phasemask_total_index_crop'

directory_gaussian = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\phasemask_total_index_crop'
directory_vortex = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\phasemask_preprocessed'

indices_vortex = extract_indices_from_filenames(directory_vortex)
indices_gaussian = extract_indices_from_filenames(directory_gaussian)


print("Length of vortex", len(indices_vortex))
print("Length of gaussian", len(indices_gaussian))

indices_both = list(set(indices_vortex) & set(indices_gaussian))
print("Indices in both arrays:", len(indices_both))

Length of vortex 20653
Length of gaussian 20269
Indices in both arrays: 20192


Clean the dataset so both gaussian and vortex have only the same coeff

In [3]:
import os
import re

def delete_non_matching_files(origin_dirs, indices_both):
    for category, origin_dir in origin_dirs.items():
        for filename in os.listdir(origin_dir):
            match = re.search(r'(\d+)_idx', filename)
            if match:
                index = int(match.group(1))
                if index not in indices_both:
                    os.remove(os.path.join(origin_dir, filename))

origin_dirs = {
    'camera_1': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\Dataset_unwrapped\camera_1_total_index_back_crop',
    'camera_2': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\Dataset_unwrapped\camera_2_total_index_back_crop',
    'phasemask': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\Dataset_unwrapped\phasemask_unwrapped_index_crop'
}

delete_non_matching_files(origin_dirs, indices_both)

print("Done.")


Done.


Create Dataset and shuffle it

In [1]:
import os
import re
import shutil
import glob
from random import Random

# Define split ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15
assert TRAIN_RATIO + VAL_RATIO + TEST_RATIO == 1.0, 'Invalid split sizes selected!'

TARGET_FOLDER = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Dataset_Schramberg_Vortex_Sven'
ORIGIN_DIRS = {
   'beam_nf': r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\camera_1_noise_crop',
   'beam_ff': r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\camera_2_noise_crop',
   'phasemask': r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\phase_unwrapped_index_crop'
}

# TARGET_FOLDER = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Gaussian_shuffle_cleaned'
# ORIGIN_DIRS = {
#    'beam_nf': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\camera_1_total_index_back_crop_cleaned',
#    'beam_ff': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\camera_2_total_index_back_crop_cleaned',
#    'phasemask': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\phasemask_total_index_crop_cleaned'
# }

# TARGET_FOLDER = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Gaussian_real_ditzingen_Abb08_shuffle_cleaned_unwrapped'
# ORIGIN_DIRS = {
#    'beam_nf': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\Dataset_unwrapped\camera_1_total_index_back_crop',
#    'beam_ff': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\Dataset_unwrapped\camera_2_total_index_back_crop',
#    'phasemask': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Gaussian_Abb08\Dataset_unwrapped\phasemask_unwrapped_index_crop'
# }

#TARGET_FOLDER = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Vortex_shuffle_cleaned'
#ORIGIN_DIRS = {
#   'beam_nf': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08\camera_1_total_index_noise_crop_cleaned',
#   'beam_ff': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08\camera_2_total_index_noise_crop_cleaned',
#   'phasemask': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08\phasemask_total_index_crop_cleaned'
#}


# TARGET_FOLDER = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Vortex_20000imgs_shuffle_cleaned_v2_unwrapped'
# ORIGIN_DIRS = {
#   'beam_nf': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\camera_1_crop_cleaned_unwrapped',
#   'beam_ff': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\camera_2_crop_cleaned_unwrapped',
#   'phasemask': r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\phasemask_unwrapped_index_crop'
# }

subdirs = ['train', 'val', 'test']
categories = ['beam_ff', 'beam_nf', 'phasemask']
for subdir in subdirs:
    for category in categories:
        os.makedirs(os.path.join(TARGET_FOLDER, subdir, category), exist_ok=True)

# suffle the data between train test val innerhalb von train müssen die indexe aber gleich sein phasemask, beam_nf, beam_ff müssen die gleichen Indexe haben!
def create_dataset():
    all_files = {category: sorted(glob.glob(os.path.join(path, '*'))) for category, path in ORIGIN_DIRS.items()}
    
    file_indices = list(range(len(all_files['beam_nf'])))
    Random(42).shuffle(file_indices)

    num_files = len(file_indices)
    train_end = int(num_files * TRAIN_RATIO)
    val_end = train_end + int(num_files * VAL_RATIO)
    
    splits = {
        'train': file_indices[:train_end],
        'val': file_indices[train_end:val_end],
        'test': file_indices[val_end:]
    }

    for split, indices in splits.items():
        for category in categories:
            for index in indices:
                src_file = all_files[category][index]
                dest_file = os.path.join(TARGET_FOLDER, split, category, os.path.basename(src_file))
                shutil.copyfile(src_file, dest_file)

create_dataset()


In [14]:
import os
import shutil
import glob
from random import Random

# Define split ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15
assert TRAIN_RATIO + VAL_RATIO + TEST_RATIO == 1.0, 'Invalid split sizes selected!'

TARGET_FOLDER = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Dataset_Schramberg_Vortex_Sven_TEST'
ORIGIN_DIRS = {
    'beam_nf': r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\camera_1_noise_crop',
    'beam_ff': r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\camera_2_noise_crop',
    'phasemask': r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\phase_unwrapped_index_crop'
}

subdirs = ['train', 'val', 'test']
categories = ['beam_ff', 'beam_nf', 'phasemask']

# Create directories for the splits and categories
for subdir in subdirs:
    for category in categories:
        os.makedirs(os.path.join(TARGET_FOLDER, subdir, category), exist_ok=True)

# Function to extract the common index from a filename
def extract_index(filename):
    # Assuming file names start with the index, e.g., '1_idx_...'
    return filename.split('_')[0]

def create_dataset():
    # Gather all files for each category
    all_files = {category: sorted(glob.glob(os.path.join(path, '*'))) for category, path in ORIGIN_DIRS.items()}

    # Create a dictionary to store the files by their common index
    indexed_files = {}
    for category, files in all_files.items():
        for file in files:
            filename = os.path.basename(file)
            index = extract_index(filename)
            if index not in indexed_files:
                indexed_files[index] = {}
            indexed_files[index][category] = file

    # Filter out indices that don't have files in all categories
    valid_indices = [index for index, file_dict in indexed_files.items() if len(file_dict) == len(categories)]

    # Shuffle the valid indices
    Random(42).shuffle(valid_indices)

    # Split indices based on the given train/val/test ratios
    num_files = len(valid_indices)
    train_end = int(num_files * TRAIN_RATIO)
    val_end = train_end + int(num_files * VAL_RATIO)
    
    splits = {
        'train': valid_indices[:train_end],
        'val': valid_indices[train_end:val_end],
        'test': valid_indices[val_end:]
    }

    # Copy the files into the respective split directories
    for split, indices in splits.items():
        for index in indices:
            for category in categories:
                src_file = indexed_files[index][category]
                dest_file = os.path.join(TARGET_FOLDER, split, category, os.path.basename(src_file))
                shutil.copyfile(src_file, dest_file)

#create_dataset()



## Simulated Data

In [ ]:
"""
The Origin folder must contain a beam folder with all NEarfield and Farfiel images and a folder with all Phasemasks.cd
"""

In [3]:
import glob
import shutil
import os
import numpy as np

# Load shuffled indices
shuffle_file = r'C:\Users\burckhardsv\Lokale_Dateien\Playground.schwartzma\Beam_Shaping_using_ML\experimental_sven\preprocessing\index_shuffle_order.npy'
shuffled_indices = np.load(shuffle_file)

# Define split (train / val / test) and origin and target directory
TRAIN = 0.7
VAL = 0.15
TEST = 0.15
TARGET_FOLDER = r'C:\Users\burckhardsv\Daten\Dataset_Vortex_Simulated_Unwrapped'
ORIGIN = r'C:\Users\burckhardsv\Daten\Raw_Data_Vortex_Intensity_Unwrapped'

# Check if split sizes sum up to 1
assert TRAIN + VAL + TEST == 1.0, 'Invalid split sizes selected!'

# Create target folder(s)
subdirs = ['train', 'val', 'test']
categories = ['beam_ff', 'beam_nf', 'phasemask']
for subdir in subdirs:
    for category in categories:
        dir_path = os.path.join(TARGET_FOLDER, subdir, category)
        os.makedirs(dir_path, exist_ok=True)

def extract_fileindex(filename):
    filename_short = filename.split('\\')[-1].split('/')[-1].split('.')[0]
    return filename_short[:-2] if filename_short[-2] == 'd' else filename_short[:-1]

# Collect files
files = list(glob.glob(ORIGIN + '/beam/*'))
file_indices = list(set([extract_fileindex(file) for file in files]))
n_instances = len(file_indices)
n_digits = len(str(n_instances))  # required for logging
assert len(files) == 2 * n_instances, 'Your "beam" folder does not meet the requirements!'

# Calculate how many instances to put into which set
n_instances_train = round(TRAIN * n_instances)
n_instances_val = round(VAL * n_instances)

# Loop through shuffled files
for i, fileindex in enumerate(shuffled_indices):

    idx = i + 1  # index is prefixed into the target filename to ensure shuffling

    # Select subset (train / test / val)
    if idx <= n_instances_train:
        subset = 'train'
    elif idx <= n_instances_train + n_instances_val:
        subset = 'val'
    else:
        subset = 'test'
    
    try:
        # Copy files with nf and ff switched
        shutil.copyfile(f'{ORIGIN}/beam/{fileindex}df.png', f'{TARGET_FOLDER}/{subset}/beam_ff/idx-{idx}__img-{fileindex}.png')
        shutil.copyfile(f'{ORIGIN}/beam/{fileindex}f.png', f'{TARGET_FOLDER}/{subset}/beam_nf/idx-{idx}__img-{fileindex}.png')
        #shutil.copyfile(f'{ORIGIN}/phasemask/Phase{fileindex}.png', f'{TARGET_FOLDER}/{subset}/phasemask/idx-{idx}__img-{fileindex}.png') # old
        shutil.copyfile(f'{ORIGIN}/phasemask/{fileindex}.png', f'{TARGET_FOLDER}/{subset}/phasemask/idx-{idx}__img-{fileindex}.png') # new
        # Show status
        if i < 10 or (i % 10 == 0 and i < 50) or (i % 100 == 0 and i < 500) or (i % 1000 == 0):
            print(f'[Completed: {idx:{n_digits}} / {n_instances}] Copying instance "{fileindex}" to "{subset}/idx-{idx}__img-{fileindex}.png"')
    
    except Exception as e:
        print(f'ERROR: Could not copy file {fileindex} (new index: {idx}): {e}')


[Completed:     1 / 20000] Copying instance "10651" to "train/idx-1__img-10651.png"
[Completed:     2 / 20000] Copying instance "2042" to "train/idx-2__img-2042.png"
[Completed:     3 / 20000] Copying instance "8669" to "train/idx-3__img-8669.png"
[Completed:     4 / 20000] Copying instance "1115" to "train/idx-4__img-1115.png"
[Completed:     5 / 20000] Copying instance "13903" to "train/idx-5__img-13903.png"
[Completed:     6 / 20000] Copying instance "11964" to "train/idx-6__img-11964.png"
[Completed:     7 / 20000] Copying instance "11073" to "train/idx-7__img-11073.png"
[Completed:     8 / 20000] Copying instance "3003" to "train/idx-8__img-3003.png"
[Completed:     9 / 20000] Copying instance "19772" to "train/idx-9__img-19772.png"
[Completed:    10 / 20000] Copying instance "8116" to "train/idx-10__img-8116.png"
[Completed:    11 / 20000] Copying instance "3526" to "train/idx-11__img-3526.png"
[Completed:    21 / 20000] Copying instance "9924" to "train/idx-21__img-9924.png"
[Co

## Test indices in a dataset, all indiceies should be the same in the folders.

In [13]:
import os
import re

def extract_indices_from_filenames(directory):
    indices = []
    pattern = re.compile(r'(\d+)_idx')

    for filename in os.listdir(directory):
        match = pattern.search(filename)
        if match:
            index = int(match.group(1))
            indices.append(index)

    return indices

directory_gaussian = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Dataset_Schramberg_Vortex_Sven_Test\train\beam_ff'
directory_vortex = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Dataset_Schramberg_Vortex_Sven_Test\train\phasemask'

indices_vortex = extract_indices_from_filenames(directory_vortex)
indices_gaussian = extract_indices_from_filenames(directory_gaussian)

print("Length of vortex", len(indices_vortex))
print("Length of gaussian", len(indices_gaussian))

# Berechnung der Indizes, die nur in einer der Listen vorkommen
indices_not_in_both = list(set(indices_vortex).symmetric_difference(set(indices_gaussian)))
print("Indices not in both arrays:", indices_not_in_both)

Length of vortex 9634
Length of gaussian 9634
Indices not in both arrays: []
